In [4]:
import json
from utils import title_to_filename


with open("qasper_dblp_merged_cleaned.json", "r") as file:
    qasper = json.load(file)

In [6]:
lookup_files = [] 
for paper in qasper:
    lookup_files.append(f"{title_to_filename(paper['title'])}.pdf")

In [7]:
import os
import shutil

# Paths (update these accordingly)
source_folder = 'intersection/'
destination_folder = 'pdfs/'

lookup_files = sorted(lookup_files)


# Ensure destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Iterate over files in source folder
for filename in os.listdir(source_folder):
    if filename in lookup_files:
        src_path = os.path.join(source_folder, filename)
        dst_path = os.path.join(destination_folder, filename)
        shutil.move(src_path, dst_path)
        print(f"Moved: {filename}")

    

Moved: what_gets_echoed_understanding_the_pointers_in_explanations_of_persuasive_arguments.pdf
Moved: detecting_online_hate_speech_using_context_aware_models.pdf
Moved: few_shot_natural_language_generation_for_task_oriented_dialog.pdf
Moved: marrying_universal_dependencies_and_universal_morphology.pdf
Moved: dens_a_dataset_for_multi_class_emotion_analysis.pdf
Moved: bertram_improved_word_embeddings_have_big_impact_on_contextualized_model_performance.pdf
Moved: polysemy_detection_in_distributed_representation_of_word_sense.pdf
Moved: synchronising_audio_and_ultrasound_by_learning_cross_modal_embeddings.pdf
Moved: stranse_a_novel_embedding_model_of_entities_and_relationships_in_knowledge_bases.pdf
Moved: efficient_twitter_sentiment_classification_using_subjective_distant_supervision.pdf
Moved: logic_attention_based_neighborhood_aggregation_for_inductive_knowledge_graph_embedding.pdf
Moved: revisiting_low_resource_neural_machine_translation_a_case_study.pdf
Moved: analysing_coreference_in

In [9]:
import pandas as pd


df = pd.read_csv("rag_responses.csv")

In [10]:
len(df)

146